See the rules here: https://docs.lammps.org/2001/data_format.html

Blank lines are important. After the header section, new entries are separated by blank lines.

Indentation and space between words/numbers on one line is not important except that keywords (e.g. Masses, Bond Coeffs) must be left-justified and capitalized as shown.

The header section (thru box bounds) must appear first in the file, the remaining entries (Masses, various Coeffs, Atoms, Bonds, etc) can come in any order.

These entries must be in the file: header section, Masses, Atoms.

Atoms format: `N molecule-tag atom-type q x y z nx ny nz`

In [1]:
function atomic2full(input_atomic, output_full)
    lines = open(input_atomic) do file; readlines(file); end

    # Get the number of the line "Atoms # atomic"
    idx = findfirst(x -> startswith(x, "Atoms"), lines)

    # Number of atoms
    n_atoms = parse(Int, filter(x -> '0' <= x <= '9', lines[3]))

    # Indices of the first and the last lines with atomic coordinates
    ib = idx + 2; ie = idx + 2 + (n_atoms - 1);

    # Array of z coordinates
    z_coords = Array{Float64}(undef, n_atoms)
    z_coord_position = 5
    for (iz, iline) ∈ enumerate(ib:ie)
        z_coords[iz] = parse(Float64, (split(lines[iline])[z_coord_position]))
    end

    # Unique z values up to two digits after the decimal point
    z_uniques = unique(trunc.(z_coords, digits = 2))
    sort!(z_uniques)

    q = "0.0" # charge

    for (iz, iline) ∈ enumerate(ib:ie)
        molecule_tag = findfirst(x -> isapprox(x, z_coords[iz]; atol=0.01), z_uniques) 
        line_vec = split(lines[iline])
        insert!(line_vec, 3, q)
        insert!(line_vec, 2, string(molecule_tag))
        lines[iline] = "      "*join(line_vec, "      ")
    end

    lines[idx] = replace(lines[idx], "atomic" => "full")
    
    open(output_full, "w") do file
        for line ∈ lines
            write(file, line*"\n") 
        end
    end    
end;

--------------------------

In [6]:
input_atomic = "hBN//hBN_bilayer_100_100.lmp";
output_full =  "hBN//hBN_bilayer_full_100_100.lmp";

In [7]:
atomic2full(input_atomic, output_full)